# GEO数据库的数据处理

## 下载文件

In [9]:
import requests

# GEO数据库实验的Accession ID
accession_id = "GSE12417"  # 请替换为你要下载的实验的Accession ID

# 构建Series Matrix文件的下载链接
series_matrix_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE18388"

# 发送HTTP请求下载Series Matrix文件
response = requests.get(series_matrix_url)

# 检查响应状态码，200表示成功
if response.status_code == 200:
    # 指定保存的文件名
    file_name = f"{accession_id}.txt"
    
    # 写入文件
    with open(file_name, 'wb') as f:
        f.write(response.content)
    print(f"文件 '{file_name}' 下载成功。")
else:
    print("文件下载失败。")


文件 'GSE12417.txt' 下载成功。


## 提取出表达矩阵

In [15]:
# 指定Series Matrix文件的路径
file_path = "/Users/guojiayi/GSE18388_series_matrix.txt"  # 请替换为你的文件路径

# 打开文件并读取表达矩阵数据
with open(file_path, 'r') as file:
    lines = file.readlines()

# 从第20行开始是表达矩阵数据
expression_data = []
for line in lines[66:]:
    # 假设每行的数据以制表符分隔
    data = line.strip().split('\t')
    # 第一个元素是gene_symbol，之后的元素是表达值
    gene_symbol = data[0]
    expression_values = list(map(float, data[1:]))
    expression_data.append((gene_symbol, expression_values))

# 输出前5行表达矩阵数据
for gene, values in expression_data[:5]:
    print(f"Gene: {gene}, Expression Values: {values}")


Gene: 10338001, Expression Values: [11.71150504, 11.6649732, 11.92198125, 11.53058547, 11.56184388, 11.91622762, 11.38164452, 11.45750899]
Gene: 10338002, Expression Values: [6.273491332, 6.438454077, 6.012187956, 6.529641458, 6.49010158, 6.444351316, 6.401128246, 6.416001061]
Gene: 10338003, Expression Values: [10.31323037, 10.36871109, 10.63295717, 10.13591981, 10.22610827, 10.67998187, 10.02053086, 10.10665246]
Gene: 10338004, Expression Values: [9.232859751, 9.265787374, 9.475008419, 9.125303512, 9.294233243, 9.510278492, 8.970523792, 9.113074867]
Gene: 10338005, Expression Values: [1.885374449, 2.09539317, 2.009620764, 2.070795684, 2.058987659, 2.101411886, 2.043781576, 2.071245021]


In [16]:
expression_data = expression_data[:35557]
split_data = [[item[0]] + item[1] for item in expression_data]

## 进一步处理表达矩阵

In [17]:
import pandas as pd
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns


将不同的探针分为不同的列

In [19]:
expression_data = expression_data[:35557]
split_data = [[item[0]] + item[1] for item in expression_data]
split_data

[['10338001',
  11.71150504,
  11.6649732,
  11.92198125,
  11.53058547,
  11.56184388,
  11.91622762,
  11.38164452,
  11.45750899],
 ['10338002',
  6.273491332,
  6.438454077,
  6.012187956,
  6.529641458,
  6.49010158,
  6.444351316,
  6.401128246,
  6.416001061],
 ['10338003',
  10.31323037,
  10.36871109,
  10.63295717,
  10.13591981,
  10.22610827,
  10.67998187,
  10.02053086,
  10.10665246],
 ['10338004',
  9.232859751,
  9.265787374,
  9.475008419,
  9.125303512,
  9.294233243,
  9.510278492,
  8.970523792,
  9.113074867],
 ['10338005',
  1.885374449,
  2.09539317,
  2.009620764,
  2.070795684,
  2.058987659,
  2.101411886,
  2.043781576,
  2.071245021],
 ['10338006',
  2.083695995,
  2.320068869,
  2.191329285,
  2.341194778,
  2.329737248,
  2.308737181,
  2.29759342,
  2.309352254],
 ['10338007',
  2.33346446,
  2.653304872,
  2.455003971,
  2.618822675,
  2.629252378,
  2.612463273,
  2.669368289,
  2.580891031],
 ['10338008',
  3.131918818,
  3.479017326,
  3.13413952,
  

为每一列标注列名

In [66]:
columns = ["ID_REF","GSM458594","GSM458595","GSM458596","GSM458597","GSM458598","GSM458599","GSM458600","GSM458601"]

expression_data = pd.DataFrame(split_data,columns = columns )
expression_data = expression_data.set_index("ID_REF")

In [69]:
expression_data.head(5) ## 显示前五行


,GSM458594,GSM458595,GSM458596,GSM458597,GSM458598,GSM458599,GSM458600,GSM458601
ID_REF,,,,,,,,
10338001,11.711505,11.664973,11.921981,11.530585,11.561844,11.916228,11.381645,11.457509
10338002,6.273491,6.438454,6.012188,6.529641,6.490102,6.444351,6.401128,6.416001
10338003,10.313230,10.368711,10.632957,10.135920,10.226108,10.679982,10.020531,10.106652
10338004,9.232860,9.265787,9.475008,9.125304,9.294233,9.510278,8.970524,9.113075
10338005,1.885374,2.095393,2.009621,2.070796,2.058988,2.101412,2.043782,2.071245


## 临床数据，将对照与实验组分开

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
# 读取表格文件，假设文件名为'your_file.csv'，可以根据实际情况替换为文件路径
Clinical_data = pd.read_csv('GSE18388_series_matrix.txt', index_col=0, header=None, skiprows=31, nrows=2)

Clinical_data

""
0
"!Sample_title\t""Space-Flown Thymus (FLT)-1""\t""Space-Flown Thymus (FLT)-2""\t""Space-Flown Thymus (FLT)-3""\t""Space-Flown Thymus (FLT)-4""\t""Control Thymus (AEM)-1""\t""Control Thymus (AEM)-2""\t""Control Thymus (AEM)-3""\t""Control Thymus (AEM)-4"""
"!Sample_geo_accession\t""GSM458594""\t""GSM458595""\t""GSM458596""\t""GSM458597""\t""GSM458598""\t""GSM458599""\t""GSM458600""\t""GSM458601"""


In [31]:
# 将数据转换为DataFrame，并指定列名
Clinical_data = pd.DataFrame(data=Clinical_data.values.T, columns=['Sample_ID', 'Group_list'])

In [32]:
Clinical_data

,Sample_ID,Group_list


In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rpy2
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate

<function rpy2.robjects.pandas2ri.activate()>